In [1]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts.chat import ChatPromptTemplate

In [ ]:
embedding_model = "models/text-embedding-004"
model = "gemini-2.0-flash-lite"
api_key = '' # Add your API key
file_path = 'assets/lilianweng_agent.txt'
qa_prompt_repo = "rlm/rag-prompt"

In [3]:
embeddings = GoogleGenerativeAIEmbeddings(model=embedding_model,google_api_key=api_key)

In [4]:
vector_store = InMemoryVectorStore(embeddings)



In [5]:
with open(file_path,'r',encoding='utf-8') as f:
    content = f.read()

In [6]:
doc = Document(page_content=content)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents([doc])

In [8]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)


In [9]:
# Define prompt for question-answering
prompt:ChatPromptTemplate = hub.pull(qa_prompt_repo)

c:\Users\molot\miniconda3\envs\llm\lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
llm = ChatGoogleGenerativeAI(
    model=model,
    api_key=api_key
)

In [11]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


In [12]:
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"], k=3)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [13]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [14]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is the process of breaking down a complex task into smaller, simpler steps. This can be done using techniques like Chain of Thought, which prompts the model to "think step by step." It can also be done with LLMs, task-specific instructions, or human input.
